<a href="https://colab.research.google.com/github/MicroPyscho/MicroPyscho/blob/main/ANN_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [290]:
!pip install tensorflow

In [291]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
from google.colab import drive
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from torch import nn

Load dataset and replace all +ve and -ve infinite values with NaN

In [292]:
star_df = pd.read_csv("/content/drive/MyDrive/Star_df dataset").replace([np.inf, -np.inf], np.nan)

if "Unnamed: 0" in star_df.columns:
  star_df.drop(columns =["Unnamed: 0"], inplace=True)

<ipython-input-292-4eea08218dbb>:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  star_df = pd.read_csv("/content/drive/MyDrive/Star_df dataset").replace([np.inf, -np.inf], np.nan)


In [293]:
var_sum = ['_CGVSDP046', '_CGVSDP047', '_CGVSDP048', '_CGVSDP049',
                    '_CGVSDP050', '_CGVSDP053', '_CGVSDP054', '_CGVSDP055', '_CGVSDP056', '_CGVSDP057', '_CGVSDP058']
star_df['SDG5_9&12_17'] = star_df[var_sum].sum(axis=1)
star_df.drop(columns=var_sum, inplace=True)

In [294]:
var_sum2 = ['_ENERDP062', '_ENERDP063', '_ENERDP068', "Target", "CCIndexd1", "SUS_report", "ENV_committee"]
star_df['waste_redctn_efforts'] = (star_df[var_sum2].sum(axis=1) / 7).clip(0, 1)
star_df.drop(columns=var_sum2, inplace=True)


In [295]:
star_df['_ENRRO04V'] = (star_df['_ENRRO04V'] - star_df['_ENRRO04V'].mean()) / star_df['_ENRRO04V'].std()
var_sum3 = ["_ENPIDP066", "_ENPIDP069", "_ENRRO04V", "EcoInn_score", "_ENERO24V"]
star_df['clean_tech_efrts'] = (star_df[var_sum3].sum(axis=1) / 5).clip(0, 1)
star_df.drop(columns=var_sum3, inplace=True)

In [296]:
var_sum4 = ["_SOCODP011", "_SOCOO10V", "CrisisMgt"]
star_df['anomaly_reportng_efrts'] = (star_df[var_sum4].sum(axis=1) / 3).clip(0, 1)
star_df.drop(columns=var_sum4, inplace=True)

In [297]:
star_df['_SODODP017'] = (star_df['_SODODP017'] - star_df['_SODODP017'].mean()) / star_df['_SODODP017'].std()
star_df['_SODODP019'] = (star_df['_SODODP019'] - star_df['_SODODP019'].mean()) / star_df['_SODODP019'].std()
var_sum5 = ["_SODODP019", "_SODODP017"]
star_df['women_rpresntn'] = (star_df[var_sum5].sum(axis=1) / 2).clip(0, 1)
star_df.drop(columns=var_sum5, inplace=True)

In [298]:
var_sum6= ['climatevul_capacity', 'climatevul_exposure', 'climatevul_sensitivity',
                    'climate_vul', 'climate_economic', 'climate_governance',
                    'climate_readiness', 'climate_social']
star_df['climate_preparedness'] = star_df[var_sum6].sum(axis=1)
star_df.drop(columns=var_sum6, inplace=True)

In [299]:
var_avg = ['CO2eGDP', 'CO2eGDP_PPP2017', 'CO2eGDP_PPP']
star_df['avgCO2_GDP'] = star_df[var_avg].mean(axis=1)
star_df.drop(columns=var_avg, inplace=True)

In [300]:
star_df['bal_of_tradeGDP%'] = star_df['ExportsGDP'] - star_df['ImportsGDP']
var_std = ['GDP_K2015', 'GDPg', 'GDP_percap2015', 'GDPg_percap', "MktCapGDP", "GovtDebtGDP", "bal_of_tradeGDP%", "TradeGDP"]
for var in var_std:
    star_df[f'{var}_standardized'] = (star_df[var] - star_df[var].mean()) / star_df[var].std()
star_df['gdp_prfmnce'] = star_df[[f'{var}_standardized' for var in var_std]].sum(axis=1)
star_df['gdp_prfmnce_rank'] = star_df['gdp_prfmnce'].rank(pct=True) * 100
star_df.drop(columns=var_std, inplace=True)

In [301]:
income_class_map = {'High income': 4, 'Upper middle income': 3, 'Lower middle income': 2, 'Low income': 1}
star_df['incme_class'] = star_df['ficincome'].map(income_class_map)
var_std2 = ['var', 'pvr', 'ger', 'rqr', 'rlr', 'ccr', 'hdi_score', 'incme_class', "gain"]
for var in var_std2:
  star_df[f'{var}_standardized'] = (star_df[var] - star_df[var].mean()) / star_df[var].std()
star_df['eco_soc_gov_fctrs'] = star_df[[f'{var}_standardized' for var in var_std2]].sum(axis=1)
star_df['eco_soc_gov_fctrs'] = (star_df['eco_soc_gov_fctrs'] - star_df['eco_soc_gov_fctrs'].min()) / (star_df['eco_soc_gov_fctrs'].max() - star_df['eco_soc_gov_fctrs'].min()) * 100
star_df.drop(columns=var_std2, inplace=True)

In [302]:
star_df['dvsrty&inclsn_score'] = (star_df['_TRDIRDS'] + star_df['_TRDIRIS']) / 2

star_df['total_emissns'] = star_df['_ENERDP023'] + star_df['_ENERDP123'] + star_df["_ENERO03V"]
star_df['Net_FDI_GDP'] = star_df['FDI_INFGDP'] - star_df['FDI_OUTGDP']

star_df['avg_EPS'] = (star_df['_EPS'] + star_df['_EPS2MN']) / 2
star_df['DCred2PrvSctr_GDP'] = (star_df['DCredPvtGDP'] - star_df['DCredPvtGDP'].mean()) / star_df['DCredPvtGDP'].std()
star_df['agg_net_cash_flow'] = star_df['_WC04860'] + star_df['_WC04870'] + star_df['_WC04890']
star_df['operating_profit'] = star_df['_WC01250'] - star_df['_WC01249']
star_df["abs_sales_growth"] = star_df['sgrowth'] - star_df['sicsgrowth']
star_df.rename(columns={'_WC03151': 'working_capital'}, inplace=True)
star_df.rename(columns={"_WC01001": "net_revenue"}, inplace=True)
star_df.rename(columns={"_WC01201": "R&D"}, inplace=True)
star_df.rename(columns={"_WC03501": "common_equity"}, inplace=True)
star_df.rename(columns={"_WC03998": "total_capital"}, inplace=True)
star_df.rename(columns={"_WC08106": "current_ratio"}, inplace=True)
star_df.rename(columns={"_WC08126": "invntry_hold_days"}, inplace=True)
star_df.rename(columns={"_WC08366": "net_prfit_margin"}, inplace=True)
star_df.rename(columns={"roe": "retrn_on_equity"}, inplace=True)
star_df.rename(columns={"_CGBSO19": "board_gndr_div%"}, inplace=True)
star_df.rename(columns={"_CGSCORE": "govnce_pilr_score"}, inplace=True)
star_df.rename(columns={"_ENERDP024": "dirct_cemissions"}, inplace=True)
star_df.rename(columns={"_ENERDP025": "indirct_cemissions"}, inplace=True)
star_df.rename(columns={"_ENERO27V": "scope1"}, inplace=True)
star_df.rename(columns={"_ENERO28V": "scope2"}, inplace=True)
star_df.rename(columns={"_ENRRDP033": "total_enrgy_use"}, inplace=True)
star_df.rename(columns={"_ENRRDP046": "renwbl_enrgy_use"}, inplace=True)
star_df.rename(columns={"_ENSCORE": "envrmt_pilr_score"}, inplace=True)
star_df.rename(columns={"_TRESGCS": "ESG_score"}, inplace=True)
star_df.rename(columns={"_TRESGCGVSS": "CSR_score"}, inplace=True)
star_df.rename(columns={"_TRESGENERS": "emissn_score"}, inplace=True)
star_df.rename(columns={"INTSpread": "Intrst_rate_spread"}, inplace=True)
star_df.rename(columns={"pop_score": "population"}, inplace=True)
star_df.rename(columns={"_MV": "market_value"}, inplace=True)
star_df.rename(columns={"_PI": "price_index"}, inplace=True)
star_df.rename(columns={"_RI": "return_index"}, inplace=True)
star_df.rename(columns={"SIC": "SIC_code"}, inplace=True)
star_df.rename(columns={"_ENERDP0161": "Emission_objtv"}, inplace=True)
star_df.rename(columns={"_ENERO03V": "Grenhous_emissn"}, inplace=True)
star_df.rename(columns={"_ENERO26V": "CO2Emissn_%+-"}, inplace=True)
star_df.rename(columns={"'_CGSRDP049'": "prvt0_vs_govtowned1"}, inplace=True)



#star_df['working_capital'] = star_df['_WC02201'] - star_df['_WC02999']

In [303]:
list_A = [
    'DSCD', 'ticker', 'exchange', 'WC06011', 'WC07040', '_EPS', '_EPS2MN', '_MVC',
    '_P', '_WC01051', '_WC01084', '_WC01249', '_WC01100', '_WC01250', '_WC01401',
    '_WC02001', '_WC02051', '_WC02101', '_WC02201', '_WC02501', '_WC02999',
    '_WC03101', '_WC03255', '_WC03351', '_WC03480', '_WC03995', '_WC04551',
    '_WC04601', '_WC04860', '_WC04870', '_WC04890', '_WC05201', '_WC05476',
    '_WC05502', '_WC08101', '_WC08321', '_WC08301', '_WC08316', '_WC08366',
    '_WC09504', '_WC18193', '_WC18199', '_WC18198', 'ffi5', 'cashna', 'cash2sales',
    'ppe', 'Size', 'logsales', 'logppe', 'firmage37', 'logage', 'tlta', 'tdta',
    'tdmv', 'clta', 'cata', 'apta', 'arta', 'arsale', 'apcogs', 'nettcsale', 'dep',
    'roa', 'ros', 'ebitta', 'tobinqe', 'mb', 'capexppe', 'dcashta', 'dcashsales',
    'dcashna', 'sgrowth', 'sicsgrowth', 'nwccta', 'divta', 'tri', 'divchg', 'lppe',
    'lsize', 'llogsales', 'lcfw', 'lroa', 'lroe', 'ltobinq', 'lcapexppe', 'larta',
    'lapta', 'llogemp', 'ldep', 'llogage', 'llogage2', 'lcashna', 'lcashsales',
    'lcash2ta', 'lempgrowth', 'lcapexrdta', 'lcash2sales', 'lcash2na', 'wwindex',
    'kzindex', 'saindex', 'mwail', 'zscore', 'zscoree', '_CGSRDP049', '_CGVSDP046',
    '_CGVSDP047', '_CGVSDP048', '_CGVSDP049', '_CGVSDP050', '_CGVSDP053',
    '_CGVSDP054', '_CGVSDP055', '_CGVSDP056', '_CGVSDP057', '_CGVSDP058',
    '_ENERDP033', '_ENERDP062', '_ENERO32V', '_SOCODP037', '_SODODP0151',
    '_SOEQO08V', '_TRDIRDs', '_TRDIRIS', '_TRESGS', '_TRESGENPIS', 'TCE', 'DCE',
    'INDCE', 'logTCE', 'logDCE', 'logINDCE', 'logEMScore', 'TCETA', 'DCETA',
    'ENVInitiatives', 'logENVInvScore', 'EnergyUse', 'RenEnergyUse',
    'EnergyUseSale', 'logEnergyUse', 'logResUse_score', 'logMgtScore',
    'logSHScore', 'logCSRScore', 'logCMScore', 'logHRScore', 'logPRScore',
    'GOVScore', 'SOSScore', 'ESG_Comp', 'MgersFem', 'BODDIV', 'EXEDIV',
    'SnrExecCompTA', 'SnrExecCompSale', 'CrisisMgt', 'EmissionsTrdg', 'SDG13',
    'Climate_gov', 'ENV_Prod', 'ENV_Asset', 'Eco_Prod', 'RespUse_Prod',
    'CleanTech', 'EcoInn_score', 'ENV_train', 'MgtTrg', 'ESGC', 'CO2eGDP',
    'CO2eGDP_PPP2017', 'CO2eGDP_PPP', 'FDI_OUTGDP', 'FDI_INFGDP', 'DCredPvtGDP',
    'GDP_Cur', 'GDP_K2015', 'GDPg', 'GDP_percap2015', 'GDPg_percap', 'MktCapGDP',
    'GovtDebtGDP', 'bal_of_tradeGDP%', 'INFL_conpx', 'INTPayExp', 'INTPayRev',
    'RealINT', 'gdp_score', 'var', 'pvr', 'ger', 'rqr', 'rlr', 'ccr', 'hdi_score',
    'incme_class', 'gain', 'ficincome', 'climatevul_capacity', 'climatevul_exposure',
    'climatevul_sensitivity', 'climate_vul', 'climate_economic', 'climate_governance',
    'climate_readiness', 'climate_social'
]

list_B = [
    '_CGSRDP049', '_CGVSDP047', '_CGVSDP048', '_CGVSDP049',
    '_CGVSDP050', '_CGVSDP053', '_CGVSDP054', '_CGVSDP055', '_CGVSDP056', '_CGVSDP057', '_CGVSDP058', '_ENERDP062', '_ENERDP063', '_ENERDP068', "Target", "CCIndexd1", "SUS_report", "ENV_committee", "_ENPIDP066", "_ENPIDP069", "_ENRRO04V", "EcoInn_score", "_SOCODP011", "_SOCOO10V", "CrisisMgt", "_SODODP019", "_SODODP017", 'climatevul_capacity', 'climatevul_exposure', 'climatevul_sensitivity',
                    'climate_vul', 'climate_economic', 'climate_governance',
                    'climate_readiness', 'climate_social', 'CO2eGDP', 'CO2eGDP_PPP2017', 'CO2eGDP_PPP',  'GDP_K2015', 'GDPg', 'GDP_percap2015', 'GDPg_percap', "MktCapGDP", "GovtDebtGDP", "bal_of_tradeGDP%", "TradeGDP", 'var', 'pvr', 'ger', 'rqr', 'rlr', 'ccr', 'hdi_score', 'incme_class', "gain"
]

# Remove duplicates from list_B in list_A
list_A = [variable for variable in list_A if variable not in list_B]

print(list_A)


['DSCD', 'ticker', 'exchange', 'WC06011', 'WC07040', '_EPS', '_EPS2MN', '_MVC', '_P', '_WC01051', '_WC01084', '_WC01249', '_WC01100', '_WC01250', '_WC01401', '_WC02001', '_WC02051', '_WC02101', '_WC02201', '_WC02501', '_WC02999', '_WC03101', '_WC03255', '_WC03351', '_WC03480', '_WC03995', '_WC04551', '_WC04601', '_WC04860', '_WC04870', '_WC04890', '_WC05201', '_WC05476', '_WC05502', '_WC08101', '_WC08321', '_WC08301', '_WC08316', '_WC08366', '_WC09504', '_WC18193', '_WC18199', '_WC18198', 'ffi5', 'cashna', 'cash2sales', 'ppe', 'Size', 'logsales', 'logppe', 'firmage37', 'logage', 'tlta', 'tdta', 'tdmv', 'clta', 'cata', 'apta', 'arta', 'arsale', 'apcogs', 'nettcsale', 'dep', 'roa', 'ros', 'ebitta', 'tobinqe', 'mb', 'capexppe', 'dcashta', 'dcashsales', 'dcashna', 'sgrowth', 'sicsgrowth', 'nwccta', 'divta', 'tri', 'divchg', 'lppe', 'lsize', 'llogsales', 'lcfw', 'lroa', 'lroe', 'ltobinq', 'lcapexppe', 'larta', 'lapta', 'llogemp', 'ldep', 'llogage', 'llogage2', 'lcashna', 'lcashsales', 'lcas

In [304]:
col_drop = ['DSCD', 'ticker', "_ENERDP023", "Comp_CS", "Whistleblower", "ENV_RIC", "InsideDealC", "AcctC", "BribeCorrFraudC", "WagesWorkC", "RepsMarketC", "CustomerC", "_ENERDP123", "_ENERO24V", "_ENERO25V", "logmktcap", "size", "arna", "apsale", "cfw", "_TRDIRDS", "countryname", 'exchange', 'WC06011', 'WC07040', '_EPS', '_EPS2MN', '_MVC', '_P', '_WC01051', '_WC01084', '_WC01249', '_WC01100', '_WC01250', '_WC01401', '_WC02001', '_WC02051', '_WC02101', '_WC02201', '_WC02501', '_WC02999', '_WC03101', '_WC03255', '_WC03351', '_WC03480', '_WC03995', '_WC04551', '_WC04601', '_WC04860', '_WC04870', '_WC04890', '_WC05201', '_WC05476', '_WC05502', '_WC08101', '_WC08321', '_WC08301', '_WC08316', '_WC08366', '_WC09504', '_WC18193', '_WC18199', '_WC18198', 'ffi5', 'cashna', 'cash2sales', 'ppe', 'Size', 'logsales', 'logppe', 'firmage37', 'logage', 'tlta', 'tdta', 'tdmv', 'clta', 'cata', 'apta', 'arta', 'arsale', 'apcogs', 'nettcsale', 'dep', 'roa', 'ros', 'ebitta', 'tobinqe', 'mb', 'capexppe', 'dcashta', 'dcashsales', 'dcashna', 'sgrowth', 'sicsgrowth', 'nwccta', 'divta', 'tri', 'divchg', 'lppe', 'lsize', 'llogsales', 'lcfw', 'lroa', 'lroe', 'ltobinq', 'lcapexppe', 'larta', 'lapta', 'llogemp', 'ldep', 'llogage', 'llogage2', 'lcashna', 'lcashsales', 'lcash2ta', 'lempgrowth', 'lcapexrdta', 'lcash2sales', 'lcash2na', 'wwindex', 'kzindex', 'saindex', 'mwail', 'zscore', 'zscoree', '_CGVSDP046', '_ENERDP033', '_ENERO32V', '_SOCODP037', '_SODODP0151', '_SOEQO08V', '_TRDIRDs', '_TRDIRIS', '_TRESGS', '_TRESGENPIS', 'TCE', 'DCE', 'INDCE', 'logTCE', 'logDCE', 'logINDCE', 'logEMScore', 'TCETA', 'DCETA', 'ENVInitiatives', 'logENVInvScore', 'EnergyUse', 'RenEnergyUse', 'EnergyUseSale', 'logEnergyUse', 'logResUse_score', 'logMgtScore', 'logSHScore', 'logCSRScore', 'logCMScore', 'logHRScore', 'logPRScore', 'GOVScore', 'SOSScore', 'ESG_Comp', 'MgersFem', 'BODDIV', 'EXEDIV', 'SnrExecCompTA', 'SnrExecCompSale', 'EmissionsTrdg', 'SDG13', 'Climate_gov', 'ENV_Prod', 'ENV_Asset', 'Eco_Prod', 'RespUse_Prod', 'CleanTech', 'ENV_train', 'MgtTrg', 'ESGC', 'FDI_OUTGDP', 'FDI_INFGDP', 'DCredPvtGDP', 'GDP_Cur', 'INFL_conpx', 'INTPayExp', 'INTPayRev', 'RealINT', 'gdp_score', 'ficincome']
col_drop = [variable.strip() for variable in col_drop]
# Drop the specified columns from the dataset
star_df.drop(columns=col_drop, inplace=True, errors='ignore')

# Check the updated dataset
print(star_df.head())

           ISIN  ICBIC               country  startyear  INCORP  year  \
0  AEE001901017     35  United Arab Emirates     2017.0  2003.0  2020   
1  AEN000401010     50  United Arab Emirates     2003.0  1979.0  2020   
2  AES000401019     50  United Arab Emirates     2005.0  1976.0  2020   
3  AEN000901019     50  United Arab Emirates     2005.0  1968.0  2020   
4  AEG000101010     50  United Arab Emirates     2003.0  1977.0  2020   

   market_value  price_index  return_index  net_revenue  ...  \
0      11000.00         47.8         53.83    9758283.0  ...   
1       1502.50        114.5        239.16    3776232.0  ...   
2        462.27         16.3         41.61     431627.0  ...   
3        818.06          9.4         20.15     183087.0  ...   
4        344.45         44.6         92.12     345879.0  ...   

   gain_standardized  eco_soc_gov_fctrs  dvsrty&inclsn_score  total_emissns  \
0           0.051478          76.295645                  9.5   3.608878e+06   
1           0.0514

In [305]:
star_df = star_df.loc[:, ~star_df.columns.str.endswith('_standardized')]

In [306]:
for col_name, count in star_df.count().items():
    print(f"{col_name}: {count}")

total_variables = len(star_df.columns)
print(f"Total number of variables: {total_variables}")


ISIN: 88234
ICBIC: 88234
country: 88234
startyear: 88234
INCORP: 88234
year: 88234
market_value: 88234
price_index: 88234
return_index: 88234
net_revenue: 88234
R&D: 88234
working_capital: 88234
common_equity: 88234
total_capital: 88234
current_ratio: 88234
invntry_hold_days: 88234
net_prfit_margin: 88234
SIC_code: 88234
ffi30: 88234
age: 88234
retrn_on_equity: 88234
tobinq: 88234
mktvalue: 88234
empgrowth: 88234
board_gndr_div%: 88234
govnce_pilr_score: 88234
_CGSRDP049: 88234
Emission_objtv: 88234
dirct_cemissions: 88234
indirct_cemissions: 88234
Grenhous_emissn: 88234
CO2Emissn_%+-: 88234
scope1: 88234
scope2: 88234
total_enrgy_use: 88234
_ENRRDP0341: 88234
renwbl_enrgy_use: 88234
envrmt_pilr_score: 88234
CSR_score: 88211
ESG_score: 88211
emissn_score: 88211
logENVScore: 88211
Scope3: 88211
CCIndex: 88211
Climate_incentive: 88211
INFL_GDPdeflator: 88234
Intrst_rate_spread: 88109
ImportsGDP: 88234
ExportsGDP: 88234
region: 88234
population: 88234
ai_ranking: 88234
ai_score: 88234
ai_

In [307]:
print("Type of star_df:", type(star_df))

# Check the structure of star_df
print("\nStructure of star_df:")
star_df.info()


Type of star_df: <class 'pandas.core.frame.DataFrame'>

Structure of star_df:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88234 entries, 0 to 88233
Data columns (total 74 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ISIN                    88234 non-null  object 
 1   ICBIC                   88234 non-null  int64  
 2   country                 88234 non-null  object 
 3   startyear               88234 non-null  float64
 4   INCORP                  88234 non-null  float64
 5   year                    88234 non-null  int64  
 6   market_value            88234 non-null  float64
 7   price_index             88234 non-null  float64
 8   return_index            88234 non-null  float64
 9   net_revenue             88234 non-null  float64
 10  R&D                     88234 non-null  float64
 11  working_capital         88234 non-null  float64
 12  common_equity           88234 non-null  float64
 13  total_capital

In [ ]:
log_star_df = pd.DataFrame()

# Iterate through each column in star_df
for col in star_df.columns:
    # Check if the column contains numerical data
    if np.issubdtype(star_df[col].dtypes, np.number):  # Check if the data type is numerical
        # Apply log transformation and add it to log_star_df
        log_star_df[col] = np.log(star_df[col])
    else:
        # If the column contains non-numerical data, simply copy it to log_star_df
        log_star_df[col] = star_df[col]

In [323]:
log_star_df.to_csv('/content/drive/My Drive/log_star_df.csv', index=False)

Exclude all object dtype variable in preparation for ANN and CNN model

In [321]:
log_star_df.select_dtypes(exclude=['object'])

,ICBIC_log,startyear_log,INCORP_log,year_log,market_value_log,price_index_log,return_index_log,net_revenue_log,R&D_log,working_capital_log,...,gdp_prfmnce_rank_log,eco_soc_gov_fctrs_log,dvsrty&inclsn_score_log,total_emissns_log,Net_FDI_GDP_log,avg_EPS_log,DCred2PrvSctr_GDP_log,agg_net_cash_flow_log,operating_profit_log,abs_sales_growth_log
0,3.555348,7.609367,7.602401,7.610853,9.305651,3.867026,3.985831,16.093627,-inf,13.807002,...,4.262286,4.334616,2.251292,15.098907,-1.305566,-0.621757,NaN,NaN,NaN,NaN
1,3.912023,7.602401,7.590347,7.610853,7.314886,4.740575,5.477133,15.144237,-inf,14.766859,...,4.262286,4.334616,2.251292,15.152275,-1.305566,-0.389838,NaN,NaN,NaN,-1.007754
2,3.912023,7.603399,7.588830,7.610853,6.136149,2.791165,3.728341,12.975317,-inf,12.075047,...,4.262286,4.334616,2.251292,15.152275,-1.305566,-0.810056,NaN,11.360776,NaN,NaN
3,3.912023,7.603399,7.584773,7.610853,6.706936,2.240710,3.003204,12.117717,-inf,9.858281,...,4.262286,4.334616,2.251292,15.152275,-1.305566,-1.548813,NaN,NaN,NaN,NaN
4,3.912023,7.602401,7.589336,7.610853,5.841949,3.797734,4.523092,12.753844,-inf,9.462810,...,4.262286,4.334616,2.251292,15.152275,-1.305566,-1.633901,NaN,10.359329,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88229,3.806662,7.607381,7.597898,7.611348,3.403195,2.674149,2.674838,15.419805,-inf,13.605289,...,4.094063,3.069250,3.496508,14.743562,-3.243237,-0.579818,NaN,12.107798,NaN,-2.344214
88230,3.806662,7.606387,7.597898,7.611842,5.554122,4.099332,4.147095,12.353000,-inf,NaN,...,3.738512,3.229204,3.496508,14.743562,0.127132,-1.272966,NaN,10.325056,NaN,NaN
88231,4.007333,7.606387,7.597898,7.611842,8.647920,5.697093,6.056550,15.447057,-inf,14.323712,...,3.738512,3.229204,3.496508,14.743562,0.127132,0.625938,NaN,12.456944,NaN,NaN
88232,3.806662,7.607381,7.597898,7.611842,3.403195,2.674149,2.674838,15.500939,-inf,13.858105,...,3.738512,3.229204,3.496508,14.743562,0.127132,-0.335473,NaN,13.304066,NaN,-3.852013


In [319]:
log_star_df.head()

,ISIN,ICBIC,country,startyear,INCORP,year,market_value,price_index,return_index,net_revenue,...,gdp_prfmnce_rank,eco_soc_gov_fctrs,dvsrty&inclsn_score,total_emissns,Net_FDI_GDP,avg_EPS,DCred2PrvSctr_GDP,agg_net_cash_flow,operating_profit,abs_sales_growth
0,NaN,35,NaN,2017.0,2003.0,2020,11000.00,47.8,53.83,9758283.0,...,70.972074,76.295645,9.5,3.608878e+06,0.271019,0.537000,-0.65084,-603955.0,-5411151.0,-0.192364
1,NaN,50,NaN,2003.0,1979.0,2020,1502.50,114.5,239.16,3776232.0,...,70.972074,76.295645,9.5,3.806706e+06,0.271019,0.677167,-0.65084,-48643.0,-3024848.0,0.365038
2,NaN,50,NaN,2005.0,1976.0,2020,462.27,16.3,41.61,431627.0,...,70.972074,76.295645,9.5,3.806706e+06,0.271019,0.444833,-0.65084,85886.0,-491569.0,-0.231642
3,NaN,50,NaN,2005.0,1968.0,2020,818.06,9.4,20.15,183087.0,...,70.972074,76.295645,9.5,3.806706e+06,0.271019,0.212500,-0.65084,-183671.0,-303641.0,-0.032408
4,NaN,50,NaN,2003.0,1977.0,2020,344.45,44.6,92.12,345879.0,...,70.972074,76.295645,9.5,3.806706e+06,0.271019,0.195167,-0.65084,31550.0,-552250.0,-0.212332


In [312]:
star_df.shape

(88234, 74)

Replace any residual "-" in any variable with the mean of the variable column

In [313]:
star_df.replace('-', pd.NA, inplace=True)
star_df = star_df.apply(pd.to_numeric, errors='coerce')
column_means = star_df.mean(skipna=True)
star_df.fillna(column_means, inplace=True)

print("Updated DataFrame:")
print(star_df)


Updated DataFrame:
       ISIN  ICBIC  country  startyear  INCORP  year  market_value  \
0       NaN     35      NaN     2017.0  2003.0  2020      11000.00   
1       NaN     50      NaN     2003.0  1979.0  2020       1502.50   
2       NaN     50      NaN     2005.0  1976.0  2020        462.27   
3       NaN     50      NaN     2005.0  1968.0  2020        818.06   
4       NaN     50      NaN     2003.0  1977.0  2020        344.45   
...     ...    ...      ...        ...     ...   ...           ...   
88229   NaN     45      NaN     2013.0  1994.0  2021         30.06   
88230   NaN     45      NaN     2011.0  1994.0  2022        258.30   
88231   NaN     55      NaN     2011.0  1994.0  2022       5698.28   
88232   NaN     45      NaN     2013.0  1994.0  2022         30.06   
88233   NaN     45      NaN     2011.0  1994.0  2022        781.51   

       price_index  return_index  net_revenue  ...  gdp_prfmnce_rank  \
0             47.8         53.83    9758283.0  ...         70.972074

Split the variables according to their dtypes and note them

In [314]:
obj_cols = star_df.select_dtypes(include=['object'])

int_cols = star_df.select_dtypes(include=["int", "int64", "float", "float64"])

print("Object Columns:")
print(obj_cols.columns)
print("\nInteger Columns:")
print(int_cols.columns)


Object Columns:
Index([], dtype='object')

Integer Columns:
Index(['ISIN', 'ICBIC', 'country', 'startyear', 'INCORP', 'year',
       'market_value', 'price_index', 'return_index', 'net_revenue', 'R&D',
       'working_capital', 'common_equity', 'total_capital', 'current_ratio',
       'invntry_hold_days', 'net_prfit_margin', 'SIC_code', 'ffi30', 'age',
       'retrn_on_equity', 'tobinq', 'mktvalue', 'empgrowth', 'board_gndr_div%',
       'govnce_pilr_score', '_CGSRDP049', 'Emission_objtv', 'dirct_cemissions',
       'indirct_cemissions', 'Grenhous_emissn', 'CO2Emissn_%+-', 'scope1',
       'scope2', 'total_enrgy_use', '_ENRRDP0341', 'renwbl_enrgy_use',
       'envrmt_pilr_score', 'CSR_score', 'ESG_score', 'emissn_score',
       'logENVScore', 'Scope3', 'CCIndex', 'Climate_incentive',
       'INFL_GDPdeflator', 'Intrst_rate_spread', 'ImportsGDP', 'ExportsGDP',
       'region', 'population', 'ai_ranking', 'ai_score', 'ai_govt', 'ai_tech',
       'ai_infr', 'SDG5_9&12_17', 'waste_redctn_e

Do the same for all non-object dtype to note where they fall between "int", "int64", "float", "float64"

In [315]:
obj_cols = int_cols.select_dtypes(include=['object'])

int_cols = int_cols.select_dtypes(include=["int", "int64", "float", "float64"])

print("Object Columns:")
print(obj_cols.columns)
print("\nInteger Columns:")
print(int_cols.columns)

Object Columns:
Index([], dtype='object')

Integer Columns:
Index(['ISIN', 'ICBIC', 'country', 'startyear', 'INCORP', 'year',
       'market_value', 'price_index', 'return_index', 'net_revenue', 'R&D',
       'working_capital', 'common_equity', 'total_capital', 'current_ratio',
       'invntry_hold_days', 'net_prfit_margin', 'SIC_code', 'ffi30', 'age',
       'retrn_on_equity', 'tobinq', 'mktvalue', 'empgrowth', 'board_gndr_div%',
       'govnce_pilr_score', '_CGSRDP049', 'Emission_objtv', 'dirct_cemissions',
       'indirct_cemissions', 'Grenhous_emissn', 'CO2Emissn_%+-', 'scope1',
       'scope2', 'total_enrgy_use', '_ENRRDP0341', 'renwbl_enrgy_use',
       'envrmt_pilr_score', 'CSR_score', 'ESG_score', 'emissn_score',
       'logENVScore', 'Scope3', 'CCIndex', 'Climate_incentive',
       'INFL_GDPdeflator', 'Intrst_rate_spread', 'ImportsGDP', 'ExportsGDP',
       'region', 'population', 'ai_ranking', 'ai_score', 'ai_govt', 'ai_tech',
       'ai_infr', 'SDG5_9&12_17', 'waste_redctn_e

In [316]:
int_cols

,ISIN,ICBIC,country,startyear,INCORP,year,market_value,price_index,return_index,net_revenue,...,gdp_prfmnce_rank,eco_soc_gov_fctrs,dvsrty&inclsn_score,total_emissns,Net_FDI_GDP,avg_EPS,DCred2PrvSctr_GDP,agg_net_cash_flow,operating_profit,abs_sales_growth
0,NaN,35,NaN,2017.0,2003.0,2020,11000.00,47.8,53.83,9758283.0,...,70.972074,76.295645,9.5,3.608878e+06,0.271019,0.537000,-0.650840,-603955.0,-5411151.0,-0.192364
1,NaN,50,NaN,2003.0,1979.0,2020,1502.50,114.5,239.16,3776232.0,...,70.972074,76.295645,9.5,3.806706e+06,0.271019,0.677167,-0.650840,-48643.0,-3024848.0,0.365038
2,NaN,50,NaN,2005.0,1976.0,2020,462.27,16.3,41.61,431627.0,...,70.972074,76.295645,9.5,3.806706e+06,0.271019,0.444833,-0.650840,85886.0,-491569.0,-0.231642
3,NaN,50,NaN,2005.0,1968.0,2020,818.06,9.4,20.15,183087.0,...,70.972074,76.295645,9.5,3.806706e+06,0.271019,0.212500,-0.650840,-183671.0,-303641.0,-0.032408
4,NaN,50,NaN,2003.0,1977.0,2020,344.45,44.6,92.12,345879.0,...,70.972074,76.295645,9.5,3.806706e+06,0.271019,0.195167,-0.650840,31550.0,-552250.0,-0.212332
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88229,NaN,45,NaN,2013.0,1994.0,2021,30.06,14.5,14.51,4974351.0,...,59.983113,21.525758,33.0,2.529575e+06,0.039037,0.560000,-2.016415,181280.0,-4347537.0,0.095923
88230,NaN,45,NaN,2011.0,1994.0,2022,258.30,60.3,63.25,231654.0,...,42.035383,25.259551,33.0,2.529575e+06,1.135567,0.280000,-1.992845,30487.0,-408076.0,-0.130396
88231,NaN,55,NaN,2011.0,1994.0,2022,5698.28,298.0,426.90,5111776.0,...,42.035383,25.259551,33.0,2.529575e+06,1.135567,1.870000,-1.992845,257029.0,-2626780.0,-0.147842
88232,NaN,45,NaN,2013.0,1994.0,2022,30.06,14.5,14.51,5394761.0,...,42.035383,25.259551,33.0,2.529575e+06,1.135567,0.715000,-1.992845,599629.0,-5300837.0,0.021237


#Random Forest Regression to predict AI Score

In [317]:
# Assuming int_cols is your DataFrame with integer format ai_score
A = int_cols.drop(columns=["ai_score"])
B = int_cols["ai_score"]

# Splitting the data into training and testing sets
A_train, A_test, B_train, B_test = train_test_split(A, B, test_size=0.3, random_state=42)

# Building the Random Forest regression model
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
rf_regressor.fit(A_train, B_train)

# Feature importances
feature_importances = rf_regressor.feature_importances_

# Getting the top 50 features
top_50_indices = feature_importances.argsort()[-50:][::-1]
top_50_features = A.columns[top_50_indices]
top_50_importances = feature_importances[top_50_indices]

# Plotting the top 50 features
plt.figure(figsize=(20, 12))
plt.barh(top_50_features, top_50_importances)
plt.xlabel('Feature Importance')
plt.title('Top 50 Features Using Random Forest Regression')
plt.gca().invert_yaxis()  # Invert y-axis to display most important features at the top
plt.savefig('/content/top_50_RFReg.pdf')
plt.show()

# Predicting on the test set
B_pred = rf_regressor.predict(A_test)

# Creating a DataFrame to display actual and predicted values side by side
results_df = pd.DataFrame({'Actual': B_test, 'Predicted': B_pred})
print(results_df)

ValueError: Input X contains NaN.
RandomForestRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Calculating Mean Absolute Error (MAE)
mae = mean_absolute_error(B_test, B_pred)

# Calculating Mean Squared Error (MSE)
mse = mean_squared_error(B_test, B_pred)

# Calculating R-squared (R2) score
r2 = r2_score(B_test, B_pred)

# Printing the evaluation metrics
print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("R-squared (R2) Score:", r2)


In [ ]:
# Saving evaluation metrics to a file
with open('/content/evaluation_metrics.txt', 'w') as f:
    f.write("Mean Absolute Error (MAE): {}\n".format(mae))
    f.write("Mean Squared Error (MSE): {}\n".format(mse))
    f.write("R-squared (R2) Score: {}\n".format(r2))

# Visualizing actual vs predicted values
plt.figure(figsize=(10, 6))
plt.scatter(B_test, B_pred)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Actual vs Predicted Values for Random Forest Regression')
plt.savefig('/content/actual_vs_predicted_RFReg.png')
plt.show()


#KNN Classifier to predict AI Ranking

In [ ]:
# Convert ai_ranking to integer format
int_cols['ai_ranking'] = int_cols['ai_ranking'].astype(int)

# Split the data into features (A) and target variable (B)
A = int_cols.drop(columns=["ai_ranking"])
B = int_cols["ai_ranking"]

# Split the data into training and testing sets
A_train, A_test, B_train, B_test = train_test_split(A, B, test_size=0.3, random_state=42)

# Initialize the KNN classifier
knn_classifier = KNeighborsClassifier(n_neighbors=5)  # You can adjust the number of neighbors

# Train the KNN model
knn_classifier.fit(A_train, B_train)

# Predictions
y_pred_train = knn_classifier.predict(A_train)
y_pred_test = knn_classifier.predict(A_test)

# Evaluate the model
# Accuracy
train_accuracy = accuracy_score(B_train, y_pred_train)
test_accuracy = accuracy_score(B_test, y_pred_test)

# Classification report
train_classification_report = classification_report(B_train, y_pred_train)
test_classification_report = classification_report(B_test, y_pred_test)

print(f'Train Accuracy: {train_accuracy}')
print(f'Test Accuracy: {test_accuracy}')

print("Train Classification Report:")
print(train_classification_report)

print("Test Classification Report:")
print(test_classification_report)


In [ ]:
metrics_filename = '/content/evaluation_metrics.txt'
classification_report_train_filename = '/content/classification_report_train.txt'
classification_report_test_filename = '/content/classification_report_test.txt'

with open(metrics_filename, 'w') as f:
    f.write(f'Train Accuracy: {train_accuracy}\n')
    f.write(f'Test Accuracy: {test_accuracy}\n')

with open(classification_report_train_filename, 'w') as f:
    f.write("Train Classification Report:\n")
    f.write(train_classification_report)

with open(classification_report_test_filename, 'w') as f:
    f.write("Test Classification Report:\n")
    f.write(test_classification_report)

# Zip the files
import zipfile
with zipfile.ZipFile('/content/evaluation_metrics_and_reports.zip', 'w') as zipf:
    for file in [metrics_filename, classification_report_train_filename, classification_report_test_filename]:
        zipf.write(file, os.path.basename(file))

In [ ]:
# Create a DataFrame with actual and predicted rankings
compard_df = pd.DataFrame({'Actual': B_test, 'Predicted': y_pred_test})

# Display the DataFrame
print(compard_df)

compard_df = pd.DataFrame({'Actual': B_test, 'Predicted': y_pred_test})

# Reset index to have numerical index
compard_df.reset_index(drop=True, inplace=True)

# Plot the actual and predicted rankings
plt.figure(figsize=(10, 6))
compard_df.plot(kind='bar', figsize=(14, 8))
plt.title('Actual vs Predicted Rankings using KNN Classifier')
plt.xlabel('Sample Index')
plt.ylabel('Ranking')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('/content/actual_predicteed_KNNC.pdf')
plt.show()

Rank the top 50 important features for predicting AI Ranking

In [ ]:
# Define number of classes
num_classes = 5  # Adjust as needed

# Discretize the target variable into classes
B_train_discretized = np.digitize(B_train, np.arange(0, 101, 100/num_classes))
B_test_discretized = np.digitize(B_test, np.arange(0, 101, 100/num_classes))

# Initialize the RandomForestClassifier
rf_classifier = RandomForestClassifier()

# Train the model
rf_classifier.fit(A_train, B_train_discretized)

# Get feature importances
feature_importances = rf_classifier.feature_importances_

# Get indices of top 50 features
top_50_indices = feature_importances.argsort()[-50:][::-1]

# Get top 50 feature names
top_50_features = A_train.columns[top_50_indices]

# Plot feature importance
plt.figure(figsize=(12, 8))
sns.barplot(x=feature_importances[top_50_indices], y=top_50_features)
plt.title('Top 50 Features Importance for AI Ranking Using KNN')
plt.xlabel('Feature Importance Score')
plt.ylabel('Feature')
plt.savefig('/content/feat_importance_KNNC.pdf')
plt.show()


#ANN Model Prediction for AI Score

standardize dataset for ANN

In [ ]:
std_cols = (int_cols - int_cols.mean()) / int_cols.std()

scaler = StandardScaler()
scaled_cols = scaler.fit_transform(std_cols)
sstar_df = pd.DataFrame(scaled_cols, columns=std_cols.columns)

In [ ]:
X =std_cols.drop(columns=["ai_score"])
y = std_cols["ai_score"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.3, random_state=42)

In [ ]:
model = Sequential([
    Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01), input_shape=(X_train.shape[1],)),
    Dropout(0.5),
    Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.5),
    Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    Dense(1)  # No activation function for regression
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Train model
history = model.fit(X_train, y_train, epochs=100, batch_size=36, validation_split=0.2, callbacks=[early_stopping])

# Evaluate model
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')

In [ ]:
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig('/content/loss_plot.pdf')
plt.show()

In [ ]:
y_pred = model.predict(X_test)
y_pred

In [ ]:
vy_test = y_test.values.reshape(-1, 1)
inv_ytest = scaler.inverse_transform(np.hstack((np.zeros((len(vy_test), std_cols.shape[1]-1)), vy_test)))

vy_pred = y_pred.reshape(-1, 1)
inv_ypred = scaler.inverse_transform(np.hstack((np.zeros((len(vy_pred), std_cols.shape[1]-1)), vy_pred)))

In [ ]:
y_testS = inv_ytest * int_cols['ai_score'].std() + int_cols['ai_score'].mean()
y_predS = inv_ypred * int_cols['ai_score'].std() + int_cols['ai_score'].mean()

comparison_df = pd.DataFrame({'Actual': y_testS.flatten(), 'Predicted': y_predS.flatten()})

# Step 3: Print/visualize the comparison
print(comparison_df)

import numpy as np

# Calculate difference between actual and predicted values
difference = np.abs(comparison_df['Actual'] - comparison_df['Predicted'])

# Define a color map
cmap = plt.get_cmap('coolwarm')  # Choose a colormap
norm = plt.Normalize(difference.min(), difference.max())  # Normalize the difference values
colors = cmap(norm(difference))

# compare using scatter plot
plt.scatter(comparison_df['Actual'], comparison_df['Predicted'], c=colors)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Scatterplot of Actual vs Predicted AI Score using ANN')
plt.colorbar(label='Absolute Difference')
plt.savefig('/content/scatter_plot_ANN.pdf')
plt.show()


# compare using line plot
plt.figure(figsize=(10, 5))
plt.plot(comparison_df['Actual'], label='Actual', color='blue')
plt.plot(comparison_df['Predicted'], label='Predicted', color='red')
plt.xlabel('Index')
plt.ylabel('Value')
plt.title('Lineplot of Actual vs Predicted AI Score using ANN')
plt.legend()
plt.savefig('/content/line_plot_ANN.pdf')
plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Calculate MAE
mae = mean_absolute_error(y_testS, y_predS)

# Calculate MSE
mse = mean_squared_error(y_testS, y_predS)

# Calculate RMSE
rmse = mean_squared_error(y_testS, y_predS, squared=False)

# Calculate R-squared
r_squared = r2_score(y_testS, y_predS)

print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)
print("R-squared:", r_squared)

In [ ]:
# Get the weights of the connections between the input layer and the first hidden layer
weights = model.layers[0].get_weights()[0]

# Calculate feature importance by summing the absolute weights for each feature
feature_importance = np.sum(np.abs(weights), axis=1)

# Get the indices of the top 50 features
top_50_indices = feature_importance.argsort()[-50:][::-1]

# Get the names of the top 50 features
top_50_features = X_train.columns[top_50_indices]

# Plot the feature importance
plt.figure(figsize=(15, 10))
plt.barh(range(len(top_50_features)), feature_importance[top_50_indices], align='center')
plt.yticks(range(len(top_50_features)), top_50_features)
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.title('Top 50 Features Importance for AI Score Using ANN')
plt.gca().invert_yaxis()
plt.savefig('/content/top_50_ANN.pdf')
plt.show()


#Random Forest Regression Analysis for AI Score

In [ ]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Feature importances
feature_importances = rf_model.feature_importances_

# Getting the top 50 features
top_50_indices = feature_importances.argsort()[-50:][::-1]
top_50_features = X.columns[top_50_indices]
top_50_importances = feature_importances[top_50_indices]

# Plotting the top 50 features
plt.figure(figsize=(20, 16))
plt.barh(top_50_features, top_50_importances)
plt.xlabel('Feature Importance')
plt.title('Top 50 Features')
plt.gca().invert_yaxis()  # Invert y-axis to display most important features at the top
plt.savefig('/content/RF_top_50_ANN.pdf')
plt.show()

# Predicting on the test set
y_pred = rf_model.predict(X_test)

# Creating a DataFrame to display actual and predicted values side by side
results_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print(results_df)

#KNN Regression Analysis for AI Ranking

In [ ]:
A =std_cols.drop(columns=["ai_ranking"])
B = std_cols["ai_ranking"]

A_train, A_test, B_train, B_test = train_test_split(A, B, test_size =0.3, random_state=42)

In [ ]:
print(B.shape)

In [ ]:
# Initialize the KNN regressor
knn_regressor = KNeighborsRegressor(n_neighbors=5)  # You can adjust the number of neighbors

# Train the KNN model
knn_regressor.fit(A_train, B_train)

# Predictions
y_pred_train = knn_regressor.predict(A_train)
y_pred_test = knn_regressor.predict(A_test)

# Evaluate the model
#RMSE
train_rmse = mean_squared_error(B_train, y_pred_train, squared=False)
test_rmse = mean_squared_error(B_test, y_pred_test, squared=False)

#MAE
train_mae = mean_absolute_error(B_train, y_pred_train)
test_mae = mean_absolute_error(B_test, y_pred_test)

# R-squared
train_r2 = r2_score(B_train, y_pred_train)
test_r2 = r2_score(B_test, y_pred_test)

print(f'Train RMSE: {train_rmse}')
print(f'Test RMSE: {test_rmse}')
print(f'Train MAE: {train_mae}')
print(f'Test MAE: {test_mae}')
print(f'Train R-squared: {train_r2}')
print(f'Test R-squared: {test_r2}')


In [ ]:
compare_df = pd.DataFrame({'Actual': B_test, 'Predicted': y_pred_test})
print(compare_df)

In [ ]:
import matplotlib.pyplot as plt

# Create a DataFrame with actual and predicted rankings
comparison_df = pd.DataFrame({'Actual': B_test, 'Predicted': y_pred_test})

# Reset index to have numerical index
comparison_df.reset_index(drop=True, inplace=True)

# Plot the actual and predicted rankings
plt.figure(figsize=(14, 8))

# Bar chart
plt.subplot(2, 1, 1)  # 2 rows, 1 column, plot 1
comparison_df.plot(kind='bar', ax=plt.gca())
plt.title('Actual vs Predicted Rankings')
plt.xlabel('Sample Index')
plt.ylabel('Ranking')
plt.xticks(rotation=45)

# Scatter plot with line
plt.subplot(2, 1, 2)  # 2 rows, 1 column, plot 2
plt.scatter(B_train, y_pred_train, label='Training data', color='blue', alpha=0.5)
plt.scatter(B_test, y_pred_test, label='Test data', color='red', alpha=0.5)
plt.plot([B_train.min(), B_train.max()], [B_train.min(), B_train.max()], 'k--', lw=2)  # Diagonal line
plt.xlabel('Actual values')
plt.ylabel('Predicted values')
plt.title('Actual vs Predicted values for AI Ranking Using KNN Regression')
plt.savefig('/content/top_50_KNN_Regression.pdf')
plt.legend()

plt.tight_layout()

plt.show()


In [ ]:
plt.figure(figsize=(10, 5))
plt.scatter(B_train, y_pred_train, label='Training data', color='blue', alpha=0.5)
plt.scatter(B_test, y_pred_test, label='Test data', color='red', alpha=0.5)
plt.xlabel('Actual values')
plt.ylabel('Predicted values')
plt.title('Actual vs Predicted values for KNN Model')
plt.legend()
plt.show()

In [ ]:
# Calculate the absolute difference in predictions when a feature value is perturbed
train_feature_importances = np.mean(np.abs(knn_regressor.predict(A_train) - knn_regressor.predict(A_train.values + 0.01)), axis=0)
test_feature_importances = np.mean(np.abs(knn_regressor.predict(A_test) - knn_regressor.predict(A_test.values + 0.01)), axis=0)

# Get the indices of features sorted by importance
train_sorted_indices = train_feature_importances.argsort()[::-1]
test_sorted_indices = test_feature_importances.argsort()[::-1]

# Get the top 50 features and their importance scores
top_train_features = A_train.columns[train_sorted_indices][:50]
top_train_scores = train_feature_importances[train_sorted_indices][:50]

top_test_features = A_test.columns[test_sorted_indices][:50]
top_test_scores = test_feature_importances[test_sorted_indices][:50]

# Plot the top 50 features and their importance scores
plt.figure(figsize=(15, 10))

plt.subplot(2, 1, 1)
plt.barh(top_train_features, top_train_scores, color='skyblue')
plt.xlabel('Importance Score')
plt.ylabel('Feature')
plt.title('Top 50 Features Importance - Train Data')
plt.gca().invert_yaxis()  # Invert y-axis to display in descending order

plt.subplot(2, 1, 2)
plt.barh(top_test_features, top_test_scores, color='skyblue')
plt.xlabel('Importance Score')
plt.ylabel('Feature')
plt.title('Top 50 Features Importance - Test Data')
plt.gca().invert_yaxis()  # Invert y-axis to display in descending order

plt.tight_layout()
plt.show()
